In [1]:
import numpy as np
from math import factorial
from scipy.special import legendre
import matplotlib.pyplot as pl
import matplotlib.colors as colors
from matplotlib import ticker

# Defintie van enkele matplotlibparameters
pl.rcParams['mathtext.fontset'] = 'cm'
pl.rcParams['mathtext.rm'] = 'serif'
pl.rcParams['legend.fancybox'] = False
pl.rcParams['legend.loc'] = 'upper right'
pl.rcParams['legend.numpoints'] = 2
pl.rcParams['legend.fontsize'] = 'large'
pl.rcParams['legend.framealpha'] = None
pl.rcParams['legend.scatterpoints'] = 3
pl.rcParams['legend.edgecolor'] = 'inherit'

# Definitie geassocieerde Legendre polynomen
def ass_legendre(x, l, m):

    leg = legendre(l)

    if m == 0:
        return leg(x)

    elif m > 0 and m <= l:
        der_leg = np.polyder(leg, m)
        return (1.-x**2)**(m/2.)*der_leg(x)

    else:
        raise NotImplementedError

# Definitie radiale golffuncties
def phi_nl(x, y, z, n, l, Z=1):

    rho = 1.*Z*np.sqrt(x**2 + y**2 + z**2)/n
    
    L_coeff = np.zeros(n-l)
    for k in range(n-l):
        L_coeff[k] = (-1)**k*factorial(n+l)**2/(factorial(k)*factorial(n-l-k-1)*factorial(2*l+1+k))
    
    L = np.poly1d(L_coeff[::-1])
    
    return np.sqrt(factorial(n-l-1)/(2.*n*factorial(n+l)**3))*(2.*Z/n)**(3./2)*(2.*rho)**l*np.exp(-rho)*L(2.*rho)

# Definitie sferische harmonieken
def Y_lm(x, y, z, l, m):

    if np.abs(m) > l:
        raise ValueError('|m| <= l')

    elif m >= 0:
        cos_theta = z/np.sqrt(x**2 + y**2 + z**2)
        phi = np.arctan(y/x)
        return np.sqrt((2*l+1)*factorial(l-m)/(4*np.pi*factorial(l+m)))*(-1)**m*np.exp(1j*m*phi)*ass_legendre(cos_theta, l, m)

    else:
        return (-1)**m*np.conjugate(Y_lm(x, y, z, l, -m))

# Definitie waterstofachtige golffuncties
def psi_nlm(x, y, z, n, l, m, Z=1):
    return phi_nl(x, y, z, n, l, Z=Z)*Y_lm(x, y, z, l, m)

In [3]:
# 3D waarschijnlijkheidsdichtheden
def plot_prob_dens(n, l, m, label, lim=None):

    x = np.arange(-lim, lim, 0.01)
    y = 0.
    z = np.arange(-lim, lim, 0.01)

    x_grid, z_grid = np.meshgrid(x, z)

    pl.contourf(x_grid, z_grid, np.abs(psi_nlm(x_grid, y, z_grid, n, l, m, Z=1))**2, [1e-6,3.33e-6,6.66e-6,1e-5,3.33e-5,6.66e-5,1e-4,3.33e-4,6.66e-4,1e-3,3.33e-3,6.66e-3,1e-2,3.33e-2,6.66e-2,1e-1,3.33e-1,6.66e-1,1.], cmap='gray_r', locator=ticker.LogLocator())
    pl.xlim([-lim, lim])
    pl.ylim([-lim, lim])
    pl.xticks(np.linspace(-lim, lim, 5))
    pl.yticks(np.linspace(-lim, lim, 5))
    pl.xlabel(r'$x$ $(a_0/Z)$')
    pl.ylabel(r'$z$ $(a_0/Z)$')
    pl.colorbar()
    pl.title(r'%s-orbitaal' % (label))
    #pl.savefig('3D_prob_dens_{}.pdf'.format(label), bbox_inches='tight')
    pl.show()

nlm_array = ['100', '200', '210', '320']
labels = ['1s', '2s', '2p', '3d']
lim = [16., 16., 16., 16.]
#lim = [1.5, 1.5, 8, 16]

%matplotlib inline
for i, nlm in enumerate(nlm_array):
    plot_prob_dens(int(nlm[0]), int(nlm[1]), int(nlm[2]), label=labels[i], lim=lim[i])

In [4]:
# Radiale waarschijnlijkheidsdichtheden   
def plot_radiale_prob_dens(n, l, label, ylim=None):

    r = np.arange(0., 16.8, 0.01)

    fig = pl.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    
    pl.plot(r, (phi_nl(r, 0., 0., n, l, Z=1)*r)**2, color='k', label=label)
    pl.xticks(np.arange(0., 16.1, 2.))
    pl.xlabel('$Zr/a_0$')
    pl.ylabel('$r^2 \, |\\varphi_{%s}(r)|^2$' % (label))
    pl.xlim([0., 17.0])
    pl.ylim([0., ylim])
    pl.title('%s-orbitaal' % (label))
    #pl.savefig('Radiale_prob_dens_{}.pdf'.format(label), bbox_inches='tight')
    pl.show()

nl_array = ['10', '20', '21', '30', '31', '32']
labels = ['1s', '2s', '2p', '3s', '3p', '3d']
ylim = [0.6, 0.2, 0.2, 0.12, 0.12, 0.12]

%matplotlib inline
for i, nl in enumerate(nl_array):
    plot_radiale_prob_dens(int(nl[0]), int(nl[1]), label=labels[i], ylim=ylim[i])